# Amazon SageMaker Semantic Segmentation Algorithm

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Data Preparation](#Data-Preparation)
  1. [Download data](#Download-data)
  2. [Setup data](#Setup-data)
  3. [Upload to S3](#Upload-to-S3)
4. [Training](#Training)
5. [Deployment](#Deployment)
6. [Inference](#Inference)

## Introduction

Semantic Segmentation (SS) is the task of classifying every pixel in an image with a class from a known set of labels. In contrast, [image classification](https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/imageclassification_caltech/Image-classification-fulltraining-highlevel.html) generates only one label per image and [object detection](https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/object_detection_birds/object_detection_birds.html) generates a bounding box along with the label for each object in the image. The semantic segmentation output is usually represented as different pixel values in the image. Therefore, the output is an integer matrix (or a grayscale image) with the same shape as the input image. This output image is also called a segmentation mask. With the Amazon SageMaker Semantic Segmentation algorithm, not only can you train your models with your own dataset but also use our pre-trained models for lazy initialization.

This notebook is an end-to-end example introducing the Amazon SageMaker Semantic Segmentation algorithm. In this demo, we will demonstrate how to train and host a semantic segmentation model using the fully-convolutional network ([FCN](https://arxiv.org/abs/1605.06211)) algorithm using the [Aerial imagery of Dubai obtained by MBRSC satellite](https://humansintheloop.org/semantic-segmentation-dataset/) for training. Amazon SageMaker Semantic Segmentation also provides the option of using Pyramid Scene Parsing Network ([PSP](https://arxiv.org/abs/1612.01105)) and [Deeplab-v3](https://arxiv.org/abs/1706.05587) in addition to the FCN Network. Along the way, we will also demonstrate how to construct a training dataset in the format that the training job will consume. Finally, we will demonstrate how to host and validate the trained model.

## Setup

This notebook has been updated for [SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/) (`sagemaker`) **version 2** - which introduces some breaking changes from the v1 API. See the [migration guide](https://sagemaker.readthedocs.io/en/stable/v2.html) for details.

In this first cell we'll upgrade the SageMaker SDK installation if needed, before we `import sagemaker`. You can skip this cell if your environment as already up-to-date:

In [ ]:
!pip install 'sagemaker>=2,<3'

To train the Semantic Segmentation algorithm on Amazon SageMaker, we need to setup and authenticate the use of AWS services. To begin with, we need an AWS account role with SageMaker access. This role that is used to give SageMaker access to your data in S3 can automatically be obtained from the role used to start the notebook.

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

We also need the S3 bucket that is used to store training data and the trained model artifacts. In this notebook, we use the default bucket that comes with Sagemaker. However, you can also create a bucket and use that bucket instead.

In [ ]:
bucket = sess.default_bucket()
prefix = "semantic-segmentation-imagery"
print(bucket)

Lastly, we need the location of the Amazon SageMaker Semantic Segmentaion algorithm's docker image, which the SDK can look up for us.

In [ ]:
training_image = sagemaker.image_uris.retrieve("semantic-segmentation", sess.boto_region_name)
print(training_image)

## Data Preparation
The Semantic Segmentation dataset has been created in collboration with the Mohammed Bin Rashid Space Center. The dataset consists of aerial imagery of Dubai obtained by MBRSC satellites and annotated with pixel-wise semantic segmentation in 6 classes. The images were segmented by the trainees of the Roia Foundation in Syria.

This semantic segmentation dataset is dedicated to the public domain by Humans in the Loop under CC0 1.0 license.
The images are densely labeled and contain the following 6 classes:

| Name        | R   | G   | B   |
| :----------:|:---:|:---:|:---:|
| Building    | 60  | 16  | 152 |
| Land        | 132 | 41  | 246 |
| Road        | 110 | 193 | 228 |
| Vegetation  | 254 | 221 | 58  |
| Water       | 226 | 169 | 41  |
| Unlabeled   | 155 | 155 | 155 |



### Download data
Let us first download the dataset.

If this notebook was run before, you may have downloaded some data and set the files up already - in which case you can skip over this section and don't need to run it all over again.

In [ ]:
%time
print("Downloading Dataset\n")
!wget https://ee-assets-prod-us-east-1.s3.us-east-1.amazonaws.com/modules/b9dd5cd5350348138876c070da03b98c/v1/imageryandmask.zip
print("Unzipping archive\n")
!unzip imageryandmask.zip -d .
print("Removing downloaded archive\n")
!rm -rf imageryandmask.zip
print("Done!\n")

### Setup data
Move the images into appropriate directory structure as described in the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/semantic-segmentation.html#semantic-segmentation-inputoutput). 

#### Steps in the data setup
* Create the data channel as per [link](https://docs.aws.amazon.com/sagemaker/latest/dg/semantic-segmentation.html#semantic-segmentation-inputoutput).
* We will not be using the label_data channel. Defaults be used
* The annotations are single-channel PNG images. The format works as long as the metadata (modes) in the image helps the algorithm read the annotation images into a single-channel 8-bit unsigned integer. 
  For more information on our support for modes, see the [Python Image Library documentation](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#modes). We recommend using the 8-bit pixel, true color P mode. 
* Use a label_to_color mapping to mapping the RGB colors for the Labels, i.e. Building, Land etc to numbers from 0-5.


In [ ]:
## Function to convert the rgb values within the numpy array to single values that represent the labels.
def img_array_to_single_val(arr, label_to_color):
    result = np.ndarray(shape=arr.shape[:2], dtype=np.uint8)
    result[:,:] = 0
    for idx, rgb in label_to_color.items():
        result[(arr==rgb).all(2)] = idx
    return result

In [ ]:
import os
import shutil
import numpy as np
from PIL import Image

root_directory="Semantic segmentation dataset"
# Directories to be created to represent the channels
os.makedirs("data/train", exist_ok=True)
os.makedirs("data/validation", exist_ok=True)
os.makedirs("data/train_annotation", exist_ok=True)
os.makedirs("data/validation_annotation", exist_ok=True)
# We wont be creating the label_map channel as we will use defaults.
os.makedirs("data/label_map", exist_ok=True)

# The source dataset does not have the data split into train and validation
# We will do a 70:30 split here for train:validation
train_txt = "trainval.txt"
train_txt_file = open(train_txt, "w")
# The maps have been created based on some images that are in true color P mode. e.g. data/train_annotation/tile2_image_part_001.png
label_to_color = {
    1: [60, 16, 152],
    2: [132, 41, 246],
    0: [110, 193, 228],
    3: [254, 221, 58],
    4: [226, 169, 41],
    5: [155, 155, 155]
}

for path, subdirs, files in os.walk(root_directory):
    dirname = path.split(os.path.sep)[-1]
    if dirname == 'images':   #Find all 'images' directories
        tile_dir=path.split(os.path.sep)[-2].lower().replace(" ","")
        images = os.listdir(path)  #List of all image names in this subdirectory
        for i, image_name in enumerate(images):  
            if image_name.endswith(".jpg"):   #Only read jpg images...
                new_file=tile_dir+"_"+image_name
                srcfile = path+"/"+image_name
                dstfile =  "data/train/"+new_file
                shutil.copyfile(srcfile, dstfile)
                train_txt_file.write(os.path.splitext(new_file)[0]+"\n")

for path, subdirs, files in os.walk(root_directory):
    dirname = path.split(os.path.sep)[-1]
    if dirname == 'masks':   #Find all 'images' directories
        tile_dir=path.split(os.path.sep)[-2].lower().replace(" ","")
        images = os.listdir(path)  #List of all image names in this subdirectory
        for i, image_name in enumerate(images):  
            if image_name.endswith(".png"):   #Only read png masks...
                new_file=tile_dir+"_"+image_name
                srcfile = path+"/"+image_name
                dstfile =  "data/train_annotation/"+new_file
                
                im = Image.open(srcfile)
                if im.mode == "RGB":
                    imarray = np.array(im)
                    label_seg = img_array_to_single_val(np.array(Image.open(srcfile)),label_to_color)
                    img_label = Image.fromarray(label_seg)
                    img_p = img_label.convert(mode = "P")
                    img_p.save(dstfile)
                else: 
                    shutil.copyfile(srcfile, dstfile)
train_txt_file.close()

### Split the dataset into Train and Validation
In total there are 72 files in this data. This number is quite low for training but enough to understand the concept and usage of the sagemaker semantic segmentation algorithm.
These 72 files are split into a 70:30 split between a train and validation dataset.

In [ ]:

import random
# Create a train and validation file listing the files that will be used
lines = open(train_txt).readlines()
count = len(lines)
random.shuffle(lines)
open('trainval.txt', 'w').writelines(lines)

with open("train.txt",'w') as file:
    for line in lines[:int(count*0.7)]:
        file.write(line)

with open("val.txt",'w') as file:
    for line in lines[int(count*0.7):]:
        file.write(line)


In [ ]:
# Move the files from data/train to data/validation 
# and
# Move the files from data/train_annotation to data/validation_annotation

import shutil

source_image_folder = "data/train"
destination_image_folder = "data/validation"
source_mask_folder = "data/train_annotation"
destination_mask_folder = "data/validation_annotation"


files_to_move = open("val.txt").readlines()

#files_to_move = ['profit.csv', 'revenue.csv']

# iterate files
for file in files_to_move:
    # construct full file path 
    sourceimg = source_image_folder + "/" + file.replace("\n", "") + ".jpg"
    destinationimg = destination_image_folder + "/" + file.replace("\n", "") + ".jpg"
    # move file
    shutil.move(sourceimg, destinationimg)

    # construct full file path
    sourcemask = source_mask_folder + "/" + file.replace("\n", "") + ".png"
    destinationmask = destination_mask_folder + "/" + file.replace("\n", "") + ".png"
    # move file
    shutil.move(sourcemask, destinationmask)   
    
    
print('Moved')

In [ ]:
#Remove the exanded archive folder

print("Removing downloaded archive & Folder")
!rm -rf Semantic\ segmentation\ dataset

Let us check if the move was completed correctly. If it was done correctly, the number of jpeg images in `train` and png images in `train_annotation` must be the same, and so in validation as well.

In [ ]:
import glob

num_training_samples = len(glob.glob1("data/train", "*.jpg"))
num_validation_samples = len(glob.glob1("data/validation", "*.jpg"))

print("Num Train Images = " + str(num_training_samples))
assert num_training_samples == len(glob.glob1("data/train_annotation", "*.png"))

print("Num Validation Images = " + str(num_validation_samples))
assert num_validation_samples == len(glob.glob1("data/validation_annotation", "*.png"))

Let us now upload our prepared datset to the S3 bucket that we decided to use in this notebook earlier. Notice the following directory structure that is used.

```bash
root 
|-train/
|-train_annotation/
|-validation/
|-validation_annotation/

```

Notice also that all the images in the `_annotation` directory are all indexed PNG files. This implies that the metadata (color mapping modes) of the files contain information on how to map the indices to colors and vice versa. Having an indexed PNG is an advantage as the images will be rendered by image viewers as color images, but the image themsevels only contain integers. The integers are also within `[0, 1 ... c-1, 255]`  for a `c` class segmentation problem, with `255` as 'hole' or 'ignore' class. We allow any mode that is a [recognized standard](https://pillow.readthedocs.io/en/3.0.x/handbook/concepts.html#concept-modes) as long as they are read as integers.

While we recommend the format with default color mapping modes such as PASCAL, the algorithm also allows users to specify their own label maps. Refer to the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/semantic-segmentation.html#semantic-segmentation-inputoutput) for more details. The label map for the PASCAL VOC dataset is the default, which is equivalent to:

```json
{
    "scale": 1
}
```

This essentially tells the algorithm to directly use the image pixel value integers as labels. Since we are using PASCAL dataset, let us create (recreate the default just for demonstration) a label map for training channel and let the algorithm use the default (which is exactly the same) for the validation channel. If `label_map` is used, please pass it to the label_map channel.


In [ ]:
#import json

#label_map = {"scale": 1}
#with open("data/train_label_map.json", "w") as lmfile:
#    json.dump(label_map, lmfile)

### Upload to S3
Let us now upload our dataset, including our optional label map.

In [ ]:
%%time
train_channel = sess.upload_data(path="data/train", bucket=bucket, key_prefix=prefix + "/train")
print(train_channel)
train_annotation_channel = sess.upload_data(
    path="data/train_annotation",
    bucket=bucket,
    key_prefix=prefix + "/train_annotation",
)
print(train_annotation_channel)
validation_channel = sess.upload_data(
    path="data/validation", bucket=bucket, key_prefix=prefix + "/validation"
)
print(validation_channel)
validation_annotation_channel = sess.upload_data(
    path="data/validation_annotation",
    bucket=bucket,
    key_prefix=prefix + "/validation_annotation",
)
print(validation_annotation_channel)
#label_map_channel = sess.upload_data(
#    path='data/train_label_map.json',
#    bucket=bucket,
#    key_prefix=prefix+'/label_map'
#)
#print(label_map_channel)

Next we need to set up an output location in S3, where the outputs of the training job (including model artifact) will be stored.

We'll use another folder in the same S3 bucket:

In [ ]:
s3_output_location = "s3://{}/{}/output".format(bucket, prefix)
print(s3_output_location)

## Training

Now we are ready to train our segmentation algorithm. To begin, let us create a `sageMaker.estimator.Estimator` object, specifying the image of the *Semantic Segmentation* algorithm container.

Here we'll also configure the base training job name (a timestamp will be appended when the job starts), and the infrastructure the training should run on (a GPU-accelerated `ml.p3.2xlarge` to keep our job nice and fast).

We'll use this Estimator later to actually start the training job in SageMaker.

In [ ]:
ss_estimator = sagemaker.estimator.Estimator(
    training_image,  # Container image URI
    role,  # Training job execution role with permissions to access our S3 bucket
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    volume_size=50,  # in GB
    max_run=360000,  # in seconds
    output_path=s3_output_location,
    base_job_name="ss-notebook-demo",
    sagemaker_session=sess,
)

In [ ]:
# Note: You can also 'attach' to a previously completed training job if, for example, you had to re-start the
# notebook and would like to skip straight to `ss_estimator.deploy()` later on:

# ss_estimator = sagemaker.estimator.Estimator.attach("ss-notebook-demo-2020-10-29-07-05-36-516")

The semantic segmentation algorithm at its core has two components.

- An encoder or backbone network,
- A decoder or algorithm network. 

The encoder or backbone network is typically a regular convolutional neural network that may or may not have had their layers pre-trained on an alternate task such as the [classification task of ImageNet images](http://www.image-net.org/). The Amazon SageMaker Semantic Segmentation algorithm comes with two choices of pre-trained or to be trained-from-scratch backbone networks ([ResNets](https://arxiv.org/abs/1512.03385) 50 or 101). 

The decoder is a network that picks up the outputs of one or many layers from the backbone and reconstructs the segmentation mask from it. Amazon SageMaker Semantic Segmentation algorithm comes with a choice of the [Fully-convolutional network (FCN)](https://arxiv.org/abs/1605.06211) or the [Pyramid scene parsing (PSP) network](https://arxiv.org/abs/1612.01105).

The algorithm also has ample options for hyperparameters that help configure the training job. The next step in our training, is to setup these networks and hyperparameters along with data channels for training the model. Consider the following example definition of hyperparameters. See the SageMaker Semantic Segmentation [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/semantic-segmentation.html) for more details on the hyperparameters.

One of the hyperparameters here for instance is the `epochs`. This defines how many passes of the dataset we iterate over and determines that training time of the algorithm. For the sake of demonstration let us run only `10` epochs. Based on our tests, train the model for `30` epochs with similar settings should give us 'reasonable' segmentation results on the Pascal VOC data. For the most part, we will stick to using the simplest of settings. For more information on the hyperparameters of this algorithm, refer to the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/segmentation-hyperparameters.html).

In [ ]:
# Setup hyperparameters
ss_estimator.set_hyperparameters(
    backbone="resnet-50",  # This is the encoder. Other option is resnet-101
    algorithm="fcn",  # This is the decoder. Other options are 'psp' and 'deeplab'
    use_pretrained_model="True",  # Use the pre-trained model.
    num_classes=6, # Number of classes in the datset
    crop_size=240,  # Size of image random crop.
    #num_classes=21,  # Pascal has 21 classes. This is a mandatory parameter.
    epochs=10,  # Number of epochs to run.
    learning_rate=0.0001,
    optimizer="rmsprop",  # Other options include 'adam', 'rmsprop', 'nag', 'adagrad'.
    lr_scheduler="poly",  # Other options include 'cosine' and 'step'.
    mini_batch_size=16,  # Setup some mini batch size.
    validation_mini_batch_size=16,
    early_stopping=True,  # Turn on early stopping. If OFF, other early stopping parameters are ignored.
    early_stopping_patience=2,  # Tolerate these many epochs if the mIoU doens't increase.
    early_stopping_min_epochs=10,  # No matter what, run these many number of epochs.
    num_training_samples=num_training_samples,  # This is a mandatory parameter, 1464 in this case.
)

Next let us define the input data "channels" for training - as defined for this algorithm in the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/semantic-segmentation.html).

Each channel can be configured either with a simple `s3://...` string pointing to the data, or by a [sagemaker.inputs.TrainingInput](https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html#sagemaker.inputs.TrainingInput) object which provides additional configuration options such as:

- How the data is distributed between nodes for multi-instance training jobs
- Whether the channel should use [Pipe Mode](https://aws.amazon.com/blogs/machine-learning/accelerate-model-training-using-faster-pipe-mode-on-amazon-sagemaker/), and other options

The channels are specified in a simple dictionary, specifying what data sources the training job should connect to and how.

In [ ]:
distribution = "FullyReplicated"

data_channels = {
    "train": sagemaker.inputs.TrainingInput(train_channel, distribution=distribution),
    "validation": sagemaker.inputs.TrainingInput(validation_channel, distribution=distribution),
    "train_annotation": sagemaker.inputs.TrainingInput(
        train_annotation_channel, distribution=distribution
    ),
    "validation_annotation": sagemaker.inputs.TrainingInput(
        validation_annotation_channel, distribution=distribution
    )
    # 'label_map': label_map_channel
}

We have our `Estimator` object with the hyperparameters and data channels for training defined, so the only remaining thing to do is actually train a model.

The following simple command will perform several steps:

- Firstly, the instances that we requested while creating the `Estimator` above are provisioned and the appropriate container image for the algorithm installed.
- Then, the data from our channels are downloaded into the instance.
- Once data and container are ready, the training job begins.
- Logs from the training will be pulled through to this notebook as it runs, including the loss on the training data (which is the pixel-wise cross-entropy loss as described in the algorithm docs) and the metrics on the validation data (pixel-wise label accuracy and mean intersection-over-union or mIoU)
- These logs and metrics will also be published to [Amazon CloudWatch](https://aws.amazon.com/cloudwatch/) - linked from the [Amazon SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/jobs) for your training job.
- When the training is complete, your model artifacts will be uploaded to Amazon S3 in the output location we specified earlier.
- As soon as the training job finishes (regardless of whether it succeeds or fails), the infrastructure is terminated - so we only pay for the GPU-accelerated training instance for the time that the job runs. Nice!

Once the job has finished a "Job complete" message will be printed below, and your trained model can then be found in the S3 location we set up for the estimator's `output_path`.

In [ ]:
ss_estimator.fit(data_channels, logs=True)

## Deployment

Once the training is done, we can deploy the trained model as an Amazon SageMaker hosted endpoint. This will allow us to make predictions (or inference) from the model.

Note that we don't have to host on the same number or type of instances that we used to train, and can choose any SageMaker-supported instance type. Training is compute-heavy job that may have different infrastructure requirements than inference/hosting. In our case we chose the GPU-accelerated `ml.p3.2xlarge` instance to train, but will host the model on a lower cost-per-hour `ml.c5.xlarge` type - because we'll only be serving occasional requests.

The endpoint deployment can be accomplished as follows:

In [ ]:
ss_predictor = ss_estimator.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

In [ ]:
# As with Estimators & training jobs, we can instead attach to an existing Endpoint:
# ss_predictor = sagemaker.predictor.Predictor("ss-notebook-demo-2020-10-29-07-23-03-086")

## Inference

Now that the trained model is deployed to an endpoint, we can use this endpoint for inference.

To test it out, let us download an image from the web which the algorithm has so-far not seen. 

In [ ]:
filename_raw = "data/train/tile2_image_part_005.jpg"

The scale of the input image may affect the prediction results and latency, so we'll down-scale the raw image before sending it to our endpoint. You could experiment with different input resolutions (and aspect ratios) and see how the results change:

In [ ]:
from matplotlib import pyplot as plt
import PIL

%matplotlib inline

filename = "data/test_resized.jpg"
width = 800

im = PIL.Image.open(filename_raw)

aspect = im.size[0] / im.size[1]

im.thumbnail([width, int(width / aspect)], PIL.Image.ANTIALIAS)
im.save(filename, "JPEG")


The endpoint accepts images in formats similar to the ones found images in the training dataset. The input `Content-Type` should be `image/jpeg`, and the output `Accept` type can be either:

- `image/png`, which produces an indexed-PNG segmentation mask as used in training: One predicted class ID per pixel... Or,
- `application/x-protobuf`, which produces a 3D matrix giving the *confidence of each class*, for each pixel.

In the SageMaker SDK, A `Predictor` has an associated **serializer** and **deserializer** which control how data gets translated to our API call, and loaded back into a Python result object.

There are pre-built [serializers](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html) and [deserializers](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html) offered by the SDK, and we're free to define custom ones so long as they offer the same API.


### Basic inference - class IDs PNG

In our first example, we'll request the simple PNG response and would like to map those into pixel arrays (assigned class for each pixel)... So we'll write a custom deserializer for that:

In [ ]:
from PIL import Image
import numpy as np


class ImageDeserializer(sagemaker.deserializers.BaseDeserializer):
    """Deserialize a PIL-compatible stream of Image bytes into a numpy pixel array"""

    def __init__(self, accept="image/png"):
        self.accept = accept

    @property
    def ACCEPT(self):
        return (self.accept,)

    def deserialize(self, stream, content_type):
        """Read a stream of bytes returned from an inference endpoint.
        Args:
            stream (botocore.response.StreamingBody): A stream of bytes.
            content_type (str): The MIME type of the data.
        Returns:
            mask: The numpy array of class labels per pixel
        """
        try:
            return np.array(Image.open(stream))
        finally:
            stream.close()


ss_predictor.deserializer = ImageDeserializer(accept="image/png")

For the input our data is already stored as a JPEG file, so we'll use the built-in `IdentitySerializer` and feed it the file bytes:

In [ ]:
ss_predictor.serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")

with open(filename, "rb") as imfile:
    imbytes = imfile.read()

# Extension exercise: Could you write a custom serializer which takes a filename as input instead?

With that configured, calling our endpoint is now simple!

In [ ]:
%%time

cls_mask = ss_predictor.predict(imbytes)

print(type(cls_mask))
print(cls_mask.shape)

Let us display the segmentation mask.

Since the raw value of each pixel is a small number (the class ID), we'll apply a [colormap](https://matplotlib.org/3.3.2/tutorials/colors/colormaps.html) to make it a bit more human readable and not just a black square!

In [ ]:
#prediction
plt.imshow(cls_mask, cmap="jet")
plt.show()
im_mask = Image.open("data/train_annotation/tile2_image_part_005.png")
#groundtruth
plt.imshow(im_mask)
plt.show()
#aerial imagery
plt.imshow(im)
plt.show()

## Delete the Endpoint

Deployed endpoints are backed by infrastructure (1x`ml.c5.xlarge` in our case, as we requested above) - so we should delete the endpoint when we're finished with it, to avoid incurring continued costs.

In [ ]:
ss_predictor.delete_endpoint()